In [10]:
df = pd.read_csv('/content/shopping_behavior_updated.csv')
df.head()

,Customer ID,Age,Gender,Item Purchased,Category,Purchase Amount (USD),Location,Size,Color,Season,Review Rating,Subscription Status,Shipping Type,Discount Applied,Promo Code Used,Previous Purchases,Payment Method,Frequency of Purchases
0,1,55,Male,Blouse,Clothing,53,Kentucky,L,Gray,Winter,3.1,Yes,Express,Yes,Yes,14,Venmo,Fortnightly
1,2,19,Male,Sweater,Clothing,64,Maine,L,Maroon,Winter,3.1,Yes,Express,Yes,Yes,2,Cash,Fortnightly
2,3,50,Male,Jeans,Clothing,73,Massachusetts,S,Maroon,Spring,3.1,Yes,Free Shipping,Yes,Yes,23,Credit Card,Weekly
3,4,21,Male,Sandals,Footwear,90,Rhode Island,M,Maroon,Spring,3.5,Yes,Next Day Air,Yes,Yes,49,PayPal,Weekly
4,5,45,Male,Blouse,Clothing,49,Oregon,M,Turquoise,Spring,2.7,Yes,Free Shipping,Yes,Yes,31,PayPal,Annually


In [11]:
# Quick look at the dataset
print("Dataset Head:")
print(df.head())

print("\nDataset Info:")
print(df.info())

print("\nSummary Statistics:")
print(df.describe())

Dataset Head:
   Customer ID  Age Gender Item Purchased  Category  Purchase Amount (USD)  \
0            1   55   Male         Blouse  Clothing                     53   
1            2   19   Male        Sweater  Clothing                     64   
2            3   50   Male          Jeans  Clothing                     73   
3            4   21   Male        Sandals  Footwear                     90   
4            5   45   Male         Blouse  Clothing                     49   

        Location Size      Color  Season  Review Rating Subscription Status  \
0       Kentucky    L       Gray  Winter            3.1                 Yes   
1          Maine    L     Maroon  Winter            3.1                 Yes   
2  Massachusetts    S     Maroon  Spring            3.1                 Yes   
3   Rhode Island    M     Maroon  Spring            3.5                 Yes   
4         Oregon    M  Turquoise  Spring            2.7                 Yes   

   Shipping Type Discount Applied Promo Co

In [16]:
import numpy as np
# Check for missing values in each column
print("\nMissing Values per Column:")
print(df.isnull().sum())


numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
for col in numeric_cols:
    if df[col].isnull().sum() > 0:
        df[col].fillna(df[col].mean(), inplace=True)


if 'Purchase Amount (USD)' in df.columns:
    df['Spending_Segment'] = pd.cut(
        df['Purchase Amount (USD)'],
        bins=[0, 100, 500, 1000, np.inf],
        labels=['Low', 'Medium', 'High', 'Very High']
    )

print("\nDataset after cleaning and new variable creation:")
print(df.head())



Missing Values per Column:
Customer ID               0
Age                       0
Gender                    0
Item Purchased            0
Category                  0
Purchase Amount (USD)     0
Location                  0
Size                      0
Color                     0
Season                    0
Review Rating             0
Subscription Status       0
Shipping Type             0
Discount Applied          0
Promo Code Used           0
Previous Purchases        0
Payment Method            0
Frequency of Purchases    0
dtype: int64

Dataset after cleaning and new variable creation:
   Customer ID  Age Gender Item Purchased  Category  Purchase Amount (USD)  \
0            1   55   Male         Blouse  Clothing                     53   
1            2   19   Male        Sweater  Clothing                     64   
2            3   50   Male          Jeans  Clothing                     73   
3            4   21   Male        Sandals  Footwear                     90   
4            5

In [17]:
# --------------------------
# 1. Frequency Mapping
# --------------------------
# Suppose "Frequency of Purchases" contains values like "Weekly", "Fortnightly", "Monthly", "Annually".
# Convert these into approximate purchase counts per year.

frequency_mapping = {
    'Weekly': 52,
    'Fortnightly': 26,
    'Monthly': 12,
    'Annually': 1
}

# Create a new column with the numeric equivalent
df['Frequency_per_Year'] = df['Frequency of Purchases'].map(frequency_mapping)

In [18]:
# --------------------------
# 2. Annual Spend & LTV
# --------------------------
# We'll assume "Purchase Amount (USD)" is the typical purchase amount for this customer
# and multiply by the frequency to get an approximate annual spend.
# Then, we assume an average customer lifespan of 5 years (just as an example).

df['Estimated_Annual_Spend'] = df['Purchase Amount (USD)'] * df['Frequency_per_Year']
df['Estimated_LTV'] = df['Estimated_Annual_Spend'] * 5  # Example: 5-year lifespan

In [19]:
df.head()

,Customer ID,Age,Gender,Item Purchased,Category,Purchase Amount (USD),Location,Size,Color,Season,...,Shipping Type,Discount Applied,Promo Code Used,Previous Purchases,Payment Method,Frequency of Purchases,Spending_Segment,Frequency_per_Year,Estimated_Annual_Spend,Estimated_LTV
0,1,55,Male,Blouse,Clothing,53,Kentucky,L,Gray,Winter,...,Express,Yes,Yes,14,Venmo,Fortnightly,Low,26.0,1378.0,6890.0
1,2,19,Male,Sweater,Clothing,64,Maine,L,Maroon,Winter,...,Express,Yes,Yes,2,Cash,Fortnightly,Low,26.0,1664.0,8320.0
2,3,50,Male,Jeans,Clothing,73,Massachusetts,S,Maroon,Spring,...,Free Shipping,Yes,Yes,23,Credit Card,Weekly,Low,52.0,3796.0,18980.0
3,4,21,Male,Sandals,Footwear,90,Rhode Island,M,Maroon,Spring,...,Next Day Air,Yes,Yes,49,PayPal,Weekly,Low,52.0,4680.0,23400.0
4,5,45,Male,Blouse,Clothing,49,Oregon,M,Turquoise,Spring,...,Free Shipping,Yes,Yes,31,PayPal,Annually,Low,1.0,49.0,245.0


# --------------------------
# 3. Interesting Grouped Insights
# --------------------------

In [20]:
# 3A. Average LTV by Subscription Status
ltv_by_subscription = df.groupby('Subscription Status')['Estimated_LTV'].mean()
print("Average LTV by Subscription Status:")
print(ltv_by_subscription)
print("-----------------------------------------------------\n")

Average LTV by Subscription Status:
Subscription Status
No     6571.767486
Yes    6682.665590
Name: Estimated_LTV, dtype: float64
-----------------------------------------------------



In [21]:
# 3B. Discount Usage Rates
discount_usage = df['Discount Applied'].value_counts(normalize=True) * 100
print("Discount Usage Rates (%):")
print(discount_usage)
print("-----------------------------------------------------\n")

Discount Usage Rates (%):
Discount Applied
No     57.0
Yes    43.0
Name: proportion, dtype: float64
-----------------------------------------------------



In [22]:
# 3C. Shipping Type Counts
shipping_type_counts = df['Shipping Type'].value_counts()
print("Shipping Type Counts:")
print(shipping_type_counts)
print("-----------------------------------------------------\n")

Shipping Type Counts:
Shipping Type
Free Shipping     675
Standard          654
Store Pickup      650
Next Day Air      648
Express           646
2-Day Shipping    627
Name: count, dtype: int64
-----------------------------------------------------



In [23]:
# 3D. Payment Method Usage by Age Group
# Create age bins for demonstration (adjust as needed).
bins = [0, 25, 40, 60, 120]  # upper bound set to 120 as a safe "max age"
labels = ['18-24', '25-39', '40-59', '60+']
df['Age Group'] = pd.cut(df['Age'], bins=bins, labels=labels, right=False)

payment_by_age = df.groupby(['Age Group', 'Payment Method']).size().unstack(fill_value=0)
print("Payment Method usage by Age Group:")
print(payment_by_age)
print("-----------------------------------------------------\n")

Payment Method usage by Age Group:
Payment Method  Bank Transfer  Cash  Credit Card  Debit Card  PayPal  Venmo
Age Group                                                                  
18-24                      83    93           81          75      77     77
25-39                     174   172          179         188     202    201
40-59                     230   259          259         251     279    232
60+                       125   146          152         122     119    124
-----------------------------------------------------



<ipython-input-23-f8cfeff88c62>:7: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  payment_by_age = df.groupby(['Age Group', 'Payment Method']).size().unstack(fill_value=0)


In [24]:
# 3E. Average Review Rating by Category
review_by_category = df.groupby('Category')['Review Rating'].mean()
print("Average Review Rating by Category:")
print(review_by_category)
print("-----------------------------------------------------\n")

Average Review Rating by Category:
Category
Accessories    3.768629
Clothing       3.723143
Footwear       3.790651
Outerwear      3.746914
Name: Review Rating, dtype: float64
-----------------------------------------------------



In [25]:
# 3F. Shipping Type vs. Discount Applied (Cross-Tab)
shipping_discount_ct = pd.crosstab(df['Shipping Type'], df['Discount Applied'])
print("Shipping Type vs. Discount Applied:")
print(shipping_discount_ct)
print("-----------------------------------------------------\n")

Shipping Type vs. Discount Applied:
Discount Applied   No  Yes
Shipping Type             
2-Day Shipping    372  255
Express           367  279
Free Shipping     397  278
Next Day Air      353  295
Standard          369  285
Store Pickup      365  285
-----------------------------------------------------



# --------------------------
# 4. Correlation Analysis
# --------------------------

In [26]:
# Look for correlations among numeric columns (e.g., Age, Purchase Amount, Review Rating, etc.)
numeric_df = df.select_dtypes(include=[np.number])
correlations = numeric_df.corr()
print("Correlation matrix among numeric columns:")
print(correlations)

Correlation matrix among numeric columns:
                        Customer ID       Age  Purchase Amount (USD)  \
Customer ID                1.000000 -0.004079               0.011048   
Age                       -0.004079  1.000000              -0.010424   
Purchase Amount (USD)      0.011048 -0.010424               1.000000   
Review Rating              0.001343 -0.021949               0.030776   
Previous Purchases        -0.039159  0.040445               0.008063   
Frequency_per_Year        -0.020078 -0.000326              -0.016732   
Estimated_Annual_Spend    -0.014438 -0.018279               0.382372   
Estimated_LTV             -0.014438 -0.018279               0.382372   

                        Review Rating  Previous Purchases  Frequency_per_Year  \
Customer ID                  0.001343           -0.039159           -0.020078   
Age                         -0.021949            0.040445           -0.000326   
Purchase Amount (USD)        0.030776            0.008063         

In [30]:
from google.colab import files

# Assuming your DataFrame is named 'df'
df.to_csv('exported_dataset.csv', index=False)  # Create a CSV file
files.download('exported_dataset.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>